<a id='top'></a>

# Watershed and Catchment Landcover Metrics

## Aspect and Percent North Covariates
Calculate aspect grid with the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid, or we can't calculate covariates at
 the watershed scale.
 * **~~aspect_rch = calculate mean aspect for stream reach (zonal statistics)~~**
 * **~~aspect_cat = calculate mean aspect over catchments (zonal statistics)~~**
 * **north_wtd = create 1/0 grid of north facing cells from aspect grid (north = aspects from 315-45 degrees), ~~~use
  weighted flow accumulation to sum north facing cells, divide by flow accumulation grid to get % of north facing
  cells in each watershed~~~ Use tabulate area to quantify area of North cells (VALUE_1) and non-North (VALUE_0) cells
  for each watershed and calculate percent north as north_wtd = row[VALUE_1]/(row[VALUE_1]+row[VALUE_0])*100 .**
## Elevation Metrics
Calculate elevation metrics for catchment/watershed with temperature data using zonal statistics as table.
* <b> Running "ALL" zonal statistics as it does not allow you to choose two types of statistics
(ex statistics_type = ["MIN","MAX"]) instead set statistics_type="ALL"</b>
### Catchment Elevation Metrics
* **cat_elev_mn = mean elevation for catchment**
* **cat_elev_min = minimum elevation for catchment**
* **cat_elev_max = max elevation for catchment**
* **cat_elev_std = standard deviation of elevation for catchment**
### Watershed Elevation Metrics
* **wtd_elev_mn = mean watershed elevation**
* **wtd_elev_min = min watershed elevation**
* **wtd_elev_max = max watershed elevation**
* **wtd_elev_sd (or cv) = standard deviation of watershed elevation**
## Slope Metrics
Calculate slope metrics for catchment/watershed with temperature data using zonal statistics as table and
statistics_type = "ALL".
### Catchment Slope Metrics
* **cat_slope_mn = mean slope for catchment**
* **cat_slope_min = minimum slope for catchment**
* **cat_slope_max = max slope for catchment**
* **cat_slope_std = standard deviation of slope for catchment**
### Watershed Elevation Metrics
* **wtd_slope_mn = mean watershed slope**
* **wtd_slope_min = min watershed slope**
* **wtd_slope_max = max watershed slope**
* **wtd_slope_sd (or cv) = standard deviation of watershed slope**
## Lake, Wetland and Glacier Cover
Lake/Pond type waterbodies from NHDPLus hydrography for those regions with NHDPlus derived datasets and NHD waterbodies
for those regions with TauDEM derived waterbodies.  Merge all waterbodies together (FTYPE = 390) and use tabulate area
calculate percent cover of lakes in catchments/watersheds.
 * **Created local copies of NHDPlus and NHD waterbodies and exported to T driver here <>  **
Use NLCD grid (wetlands.tif) and tabulate area on wetlands grid with watersheds (sum) / divide by number of cells in
watershed from fac grid, wetlands from NLCD
## Distance from coast
DO NOT WORK ON THIS


## Import modules

In [19]:
import arcpy
import os
import datetime
import time
import sys
arcpy.env.overwriteOutput = True
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

21102021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\landcover
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Collect data and set working output locations
Set data_dir to folder containing all AKSSF regional subfolders/geoadatabases
* RS data folder: data_dir = r"W:\GIS\AKSSF"
* DM data folder: data_dir = r"D:\GIS_Temp\AKSSF_BeckyCopy\AKSSF"

In [20]:
# Set AKSSF Data directory
# data_dir = r"W:\GIS\AKSSF"

# dm local
data_dir = r"D:\GIS_Temp\AKSSF"

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces(workspace_type="Folder")
rois= []
print (regions)

# Path to create output folder/gdb
temppath = r"D:\GIS_temp" # Output folder
dirname = 'AKSSF_land_met'
tempgdbname = 'AKSSF_land_met.gdb'
temp_dir = os.path.join(temppath, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

['D:\\GIS_temp\\AKSSF\\Bristol_Bay', 'D:\\GIS_temp\\AKSSF\\Cook_Inlet', 'D:\\GIS_temp\\AKSSF\\Copper_River', 'D:\\GIS_temp\\AKSSF\\Kodiak', 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']
Working Folder already created... D:\GIS_temp\AKSSF_land_met
Output location already exists D:\GIS_temp\AKSSF_land_met\AKSSF_land_met.gdb


In [21]:
# Set regional workspaces from AKSSF data folder and store in list
arcpy.env.workspace = data_dir
regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

['D:\\GIS_temp\\AKSSF\\Bristol_Bay',
 'D:\\GIS_temp\\AKSSF\\Cook_Inlet',
 'D:\\GIS_temp\\AKSSF\\Copper_River',
 'D:\\GIS_temp\\AKSSF\\Kodiak',
 'D:\\GIS_temp\\AKSSF\\Prince_William_Sound']

## Create aspect and north grids if they do not already exists
### NHDPlus

In [ ]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with NHDPlus elev data

# elrasters = []
# nhd_dat = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_NHDPlus\\extracts"
# for dirpath, dirname, filenames in arcpy.da.Walk(nhd_dat, 'RasterDataset'):
#     for filename in filenames:
#         if filename == "elev_cm.tif":
#             elras = os.path.join(dirpath, filename)
#             elrasters.append(elras)
#             print (f'Appending {elras} to list')
#
# spatial_ref = arcpy.Describe(elrasters[0]).spatialReference
# print (spatial_ref.name)
# # Merge all nhdplus elevation rasters together
# elevname = 'AKSSF_NHDPlus_elev_cm.tif'
# AKSSF_elev_cm = arcpy.MosaicToNewRaster_management(elrasters,temp_dir,elevname, spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_NHDPlus_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_cm, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_NHDPlus_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

### TauDEM - Create grids using Timms Composite DEM for entire TauDEM processed areas

In [ ]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with TauDEM elev data
# compDEM = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\data\topography\AKSSF_Elevation_Composite_10m.tif"
# clip to study area
# AKSSF_elev_10m = r"D:\\GIS_temp\\AKSSF_Composite_10m_extract.tif"
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_Composite_10m_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_10m, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_Composite_10m_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))


## Begin GIS portion
- May be faster and save more space to create aspect rasters here using extract by mask?
- May be able to use aspect raster with tabulate intersection to calculate percent North (aspects from 315-45 degrees)

### <b>UPDATE - 2021-10-12 Do not run aspect metrics at this time. Only need to run percent North for watersheds</b>

In [31]:
from arcpy.sa import *
import re

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_wet_taba_tables = []
cat_elev_ztables = []
wtd_elev_ztables = []
cat_slope_ztables = []
wtd_slope_ztables = []


# Clear lists
cat_cur_fields = []
wtd_cur_fields = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
# Test on Kodiak
rois = ['Kodiak']
for roi in rois:
    # Loop through regional folders
    for region in regions:
        if roi in str(region):
            print(f'{roi} using data from {region} folder')
            # Set data and variables unique to regions with NHDPlus Data
            if roi in nhdplus_dat:
                # # NHDPLus raster data for DM pc, variables will be overwritten below if they exist in source folder.
                # asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_aspect.tif"
                # elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_elev_cm.tif"
                # nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_NHDPlus_north_aspect.tif"
                lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond"
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
                print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                # print(f'NHD data: north raster - {nor_rast}')
                # print(f'         aspect raster - {asp_rast}')
                # print(f'      elevation raster - {elev_rast}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                # # TauDEM raster data for DM pc, variables will be overwritten below if they exist in source folder.
                # nor_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_north_aspect.tif"
                # asp_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_aspect.tif"
                # elev_rast = r"D:\Basedata\AKSSF_Basedata\AKSSF_Composite_10m_extract.tif"

                lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds"
                # Fields for update cursor
                cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]

                print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
                # print(f'TauDem data: north raster - {nor_rast}')
                # print(f'         aspect raster - {asp_rast}')
                # print(f'      elevation raster - {elev_rast}')
                print('----------')

            wtds = []
            cats = []
            # Start iter timing function
            iteration_start = time.time()
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
            for dirpath, dirnames, filenames in walk:
                # Create list of watersheds
                for filename in filenames:
                    if (("wtd" in filename) and ("merge" not in filename)):
                        print(f" {filename} watershed identified")
                        wtds.append(os.path.join(dirpath, filename))
                    # Set watershed merged watersheds dataset
                    elif 'wtds_merge'in filename:
                        wtd_merge = os.path.join(dirpath, filename)
                        print(f'Merged watershed dataset {filename} found')
                        print('----------')
                        wtdfieldnames = []
                        wtdlstFields = arcpy.ListFields(wtd_merge)
                        for field in wtdlstFields:
                            wtdfieldnames.append(field.name)
                        if str(wtd_cur_fields[0]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[0] = str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(wtd_cur_fields[2]) in wtdfieldnames:
                            print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                            print('----------')
                        else:
                            print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[2] = str(roi) +'_'+ str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

                    # Select glaciers fc
                    elif 'glaciers' in filename:
                        glac_fc = os.path.join(dirpath, filename)

                    # Select elevation raster
                    elif 'elev.tif' == filename:
                        elev_rast = os.path.join(dirpath, filename)

                    # # Select aspect raster
                    # elif 'aspect' in filename:
                    #     asp_rast = os.path.join(dirpath, filename)

                    # Select north raster
                    elif 'north.tif' == filename:
                        nor_rast = os.path.join(dirpath, filename)

                    # Select slope raster
                    elif 'slope.tif' == filename:
                        slope_rast = os.path.join(dirpath, filename)

                    # Select wetland raster
                    elif 'wetlands.tif' == filename:
                        wet_rast = os.path.join(dirpath, filename)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' in filename:
                        cats = os.path.join(dirpath,filename)
                        catlstfields = arcpy.ListFields(cats)
                        catfieldnames = []
                        for field in catlstfields:
                            catfieldnames.append(field.name)
                        if str(cat_cur_fields[0]) in catfieldnames:
                            print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[0] = str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)
                        if str(cat_cur_fields[2]) in catfieldnames:
                            print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field & cat_ID + region concat field
                            arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                            # populate cat_ID_con
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                                for row in cur:
                                    int_val = int(row[1])
                                    row[2] = str(roi) +'_'+ str(int_val)
                                    # Update rows
                                    cur.updateRow(row)
                                del(row)
                            del(cur)

            print (f'Calculating topographic metrics for catchments & watersheds of interest in {roi}'
                   f' region')
            print ('----------')
            print(f'Geodatabase: {gdb}')
            print ('----------')
            print (f'Elevation Raster: {elev_rast}')
            print ('----------')
            print (f'North Aspect Raster: {nor_rast}')
            print ('----------')
            print (f'Wetlands Raster: {wet_rast}')
            print ('----------')
            print (f'Slope Raster: {slope_rast}')
            print ('----------')        
            print (f'Lakes Ponds fc: {lakes_fc}')
            print ('----------')
            print (f'Glaciers fc: {glac_fc}')
            print ('----------')
            print (f'{len(wtds)} Watersheds to process')
            print ('----------')
            print (f'Catchment intersect {cats} selected')
            print ('----------')



            try:
                # Set output names/paths for watershed and catchment tables
                wtd_merge_name = roi + "_Watersheds_Merge"
                wtd_merge_path = os.path.join(outgdb,wtd_merge_name)

                # # Aspect variables
                # wtd_merge_asp_table_name = roi + "_Watersheds_Merge_AspectZstats"
                # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
                # cat_asp_table_name = roi + "_Catchments_AspectZstats"
                # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

                # Percent North variables
                wtd_merge_pernorth_table_name = roi + "_Watersheds_Merge_PercentNorth"
                wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
                # cat_pernorth_table_name = roi + "_Catchments_PercentNorth"
                # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

                # Elevation variables
                wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
                wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
                cat_elev_table_name = roi + "_Catchments_ElevZstats"
                cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)
                
                # Slope variables
                wtd_merge_slope_table_name = roi + "_Watershed_Merge_SlopeZstats"
                wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
                cat_slope_table_name = roi + "_Catchments_SlopeZstats"
                cat_slope_table_path = os.path.join(outgdb, cat_slope_table_name)

                # Lakes Ponds variables
                wtd_merge_lp_table_name = roi + "_Watershed_Merge_LakesPonds"
                wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
                cat_lp_table_name = roi + "_Catchments_LakesPonds"
                cat_lp_path = os.path.join(outgdb, cat_lp_table_name)

                # Wetlands variables
                wtd_merge_wetlands_table_name = roi + "_Watershed_Merge_Wetlands"
                wtd_merge_wetlands_table_path = os.path.join(outgdb, wtd_merge_wetlands_table_name)
                cat_wetlands_table_name = roi + "_Catchments_Wetlands"
                cat_wetlands_table_path = os.path.join(outgdb, cat_wetlands_table_name)

                # Glaciers
                wtd_merge_glac_table_name = roi + "_Watershed_Merge_Glaciers"
                wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)
                cat_glac_table_name = roi + "_Catchments_Glaciers"
                cat_glac_table_path = os.path.join(outgdb, cat_glac_table_name)

                # # Watersheds already merged so no need to run at this time
                # if not arcpy.Exists(wtd_merge):
                #     mergestart = time.time()
                #     # Merge watersheds
                #     wtd_merge = arcpy.Merge_management( wtds, wtd_merge_path ,add_source='ADD_SOURCE_INFO')
                #     # Add wtd_id field
                #     arcpy.AddField_management(wtd_merge,'cat_ID_txt',field_type='TEXT')
                #     arcpy.AddField_management(wtd_merge,"cat_ID_con",field_type='TEXT')
                #     # Add region field
                #     arcpy.AddField_management(wtd_merge,'region',field_type='TEXT')
                #     # Populate watershed id and region information using update cursor - faster than field calc
                #     with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','cat_ID_txt','region','cat_ID_con']) as cur:
                #         for row in cur:
                #             row[1] = re.findall('\d+', row[0])[0]
                #             row[2] = roi
                #             row[3] = + str(roi) + '_'+ str(re.findall('\d+', row[0])[0])
                #             # Update
                #             cur.updateRow(row)
                #         del(row)
                #     del(cur)
                #     mergestop = time.time()
                #     mergetime = int (mergestop - mergestart)
                #     print(f'Watershed Merge for {roi} Elapsed time: ({datetime.timedelta(seconds=mergetime)})')
                #     print('----------')
                # else:
                #     print (f'Merged watershed dataset {wtd_merge} already created')
                #     print('----------')

                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin Elevation zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                      f' region')

                # Elevation Zonal statistics  for watersheds
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                wtd_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = wtd_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = wtd_merge_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
                # Update region field
                with arcpy.da.UpdateCursor(wtd_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed elev table to list
                wtd_elev_ztables.append(wtd_elev_metrics_table)

                # Elevation zonal statistics for catchments
                arcpy.env.snapRaster = elev_rast
                arcpy.env.cellSize = elev_rast
                cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = cat_cur_fields[0],
                                                                in_value_raster = elev_rast,
                                                                out_table = cat_elev_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(cat_elev_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment elev table to list
                cat_elev_ztables.append(cat_elev_metrics_table)

                # Slope zonal statistics for catchments
                arcpy.env.snapRaster = slope_rast
                arcpy.env.cellSize = slope_rast
                cat_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                                zone_field = cat_cur_fields[0],
                                                                in_value_raster = slope_rast,
                                                                out_table = cat_slope_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_slope_metrics_table,'region',field_type='TEXT')

                # Update region field
                with arcpy.da.UpdateCursor(cat_slope_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append catchment slope table to list

                cat_slope_ztables.append(cat_slope_metrics_table)
                # Watershed slope Zonal Statistics
                print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
                      f' region')

                # Elevation Zonal statistics  for watersheds
                arcpy.env.snapRaster = slope_rast
                arcpy.env.cellSize = slope_rast
                wtd_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = wtd_cur_fields[0],
                                                                in_value_raster = slope_rast,
                                                                out_table = wtd_merge_slope_table_path,
                                                                statistics_type='ALL'
                                                                )
                # Add region identifier field for watershed tables                                                )
                arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
                # Update region field
                with arcpy.da.UpdateCursor(wtd_slope_metrics_table,'region') as cur:
                    for row in cur:
                        row[0] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed slope table to list
                wtd_slope_ztables.append(wtd_slope_metrics_table)

                # # Aspect Zonal statistics  for watersheds
                # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
                # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
                # wtd_asp_ztables.append(wtd_asp_metrics_table)

                # # Aspect Zonal statistics for catchments
                # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
                #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
                #                                                statistics_type='ALL')
                # cat_asp_ztables.append(cat_asp_metrics_table)

                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
                print('----------')

                # Tabulate Area with north grid and watersheds
                tabarea_start = time.time()
                print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
                print('----------')
                # Percent North Tabulate area for watersheds
                wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field= wtd_cur_fields[0],
                                                              in_class_data=nor_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_pernorth_table_path
                                                              )
                # Add region and percent north fields
                arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, 'wtd_north_per', field_type='Float')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
                wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
                print (wtdnorfields)
                with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
                    for row in cur:
                        row[4] = roi
                        row[5] = row[3]/(row[3]+row[2])*100
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Drop UPPERCASE field form tab area
                arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
                # Append watershed percent north table to list
                wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

                # Percent Lakes Ponds using Tabulate Intersection for watersheds
                print(f'Begin tabulate intersection between {lakes_fc} and watersheds in {roi} region')
                print('----------')
                wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                    zone_fields=wtd_cur_fields[0],
                                                                    in_class_features=lakes_fc,
                                                                    out_table=wtd_merge_lp_table_path,
                                                                    class_fields='Ftype',
                                                                    out_units="SQUARE_METERS"
                                                                    )
                # Add region and cat id fields
                arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','wtd_lake_per','wtd_lake_per')
                arcpy.AlterField_management(wtd_lp_tabint,'AREA','wtd_lake_area_sqm','wtd_lake_area_sqm')
                arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
                arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
                wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
                print (wtdlpfields)
                with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
                    for row in cur:
                        row[5] = roi
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)

                # Append watershed lakes ponds table to list
                wtd_lp_tabint_tables.append(wtd_lp_tabint)

                # Percent glaciers using Tabulate Intersection for watersheds
                print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
                print('----------')
                wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                                    zone_fields=wtd_cur_fields[0],
                                                                    in_class_features=glac_fc,
                                                                    out_table=wtd_merge_glac_table_path,
                                                                    class_fields='O1Region',
                                                                    out_units="SQUARE_METERS"
                                                                    )
                # Add region and cat id fields
                arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','wtd_glacier_per','wtd_glacier_per')
                arcpy.AlterField_management(wtd_glac_tabint,'AREA','wtd_glacier_area_sqm','wtd_glacier_area_sqm')
                arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
                arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
                wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
                print (wtdglacfields)
                with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
                    for row in cur:
                        row[5] = roi
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Append watershed percent north table to list
                wtd_glac_tabint_tables.append(wtd_glac_tabint)

                # Tabulate Area with wetlands grid and watersheds
                print(f'Begin tabulate intersection between {wet_rast} and watersheds in {roi} region')
                print('----------')
                # Percent North Tabulate area for watersheds
                wtd_per_wet_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                              zone_field= wtd_cur_fields[0],
                                                              in_class_data=wet_rast,
                                                              class_field="Value",
                                                              out_table=wtd_merge_wetlands_table_path
                                                              )
                # Add region and percent wet fields
                arcpy.AlterField_management(wtd_per_wet_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                arcpy.AddField_management(wtd_per_wet_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_wet_tabarea, 'wtd_wet_per', field_type='Float')
                arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[0], field_type='TEXT')
                arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[2], field_type='TEXT')
                wtdwetfields = [f.name for f in arcpy.ListFields(wtd_per_wet_tabarea)]
                print (wtdwetfields)
                with arcpy.da.UpdateCursor(wtd_per_wet_tabarea, wtdwetfields) as cur:
                    for row in cur:
                        row[4] = roi
                        row[5] = row[3]/(row[3]+row[2])*100
                        row[6] = row[1]
                        row[7] = roi +'_'+ row[1]
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                # Drop UPPERCASE field form tab area
                arcpy.DeleteField_management(wtd_per_wet_tabarea,'CAT_ID_TXT_DEL')
                # Append watershed percent north table to list
                wtd_wet_taba_tables.append(wtd_per_wet_tabarea)

                # # Percent North Tabulate Area for catchments
                # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_con',
                #                                             in_class_data=nor_rast,"Value",
                #                                             out_table=cat_pernorth_table_path)

                # # Add and calculate region identifier field for catchment table
                # arcpy.AlterField_management(cat_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
                # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, 'cat_north_per', field_type='Float')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[0], field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[2], field_type='TEXT')
                # catnorfields = [f.name for f in arcpy.ListFields(cat_per_north_tabarea)]
                # print (catnorfields)
                # with arcpy.da.UpdateCursor(cat_per_north_tabarea,catnorfields) as cur:
                #     for row in cur:
                #         row[4] = roi
                #         row[5] = row[3]/(row[3]+row[2])*100
                #         row[6] = row[1]
                #         row[7] = roi +'_'+ row[1]
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # Drop UPPERCASE field form tab area
                # arcpy.DeleteField_management(cat_per_north_tabarea,'CAT_ID_TXT_DEL')
                # # Append catchment percent north table to list
                # cat_pernorth_taba_tables.append(cat_per_north_tabarea)

                tabarea_stop = time.time()
                tabarea_time = int (tabarea_stop - tabarea_start)
                print(f'Tabulate area/intersections for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
                print('----------')

            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

            iter_stop = time.time()
            iter_time = int(iter_stop - iteration_start)
            print(f'All Covariates for {roi} completed. Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
            print('----------')

    else:
        print(f'Region {str(roi)} not found in {region}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



Kodiak using data from D:\GIS_temp\AKSSF\Kodiak folder
Kodiak in ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound'] TauDEM list, using cat_fields ['cat_ID_txt', 'gridcode', 'cat_ID_con'] and watershed fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
----------
cat_ID_txt field already in dataset D:\GIS_temp\AKSSF\Kodiak\Kodiak.gdb\cats_intersect
----------
cat_ID_con field already in dataset D:\GIS_temp\AKSSF\Kodiak\Kodiak.gdb\cats_intersect
----------
Merged watershed dataset wtds_merge found
----------
cat_ID_txt field already in dataset
----------
cat_ID_con field already in dataset D:\GIS_temp\AKSSF\Kodiak\Kodiak.gdb\wtds_merge
----------
 wtd_48267 watershed identified
 wtd_49617 watershed identified
 wtd_50197 watershed identified
 wtd_64593 watershed identified
 wtd_72144 watershed identified
 wtd_76954 watershed identified
 wtd_77794 watershed identified
 wtd_90346 watershed identified
 wtd_93176 watershed identified
 wtd_94216 watershed identified
 wtd_97276 watershed identified
 

## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [32]:
# Lists to store output tables
# wtd_pernorth_taba_tables=[]
# wtd_lp_tabint_tables = []
# wtd_glac_tabint_tables = []
# wtd_wet_taba_tables = []
# cat_elev_ztables = []
# wtd_elev_ztables = []
# cat_slope_ztables = []
# wtd_slope_ztables = []

# Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_cat_elev')
cat_slope_table_out = os.path.join(outgdb,'AKSSF_cat_slope')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_wtd_elev')
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_wtd_slope')
wtd_wet_table_out = os.path.join(outgdb, 'AKSSF_wtd_wetland_per')

# Merge all regional tables together
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
cat_slope = arcpy.Merge_management(cat_slope_ztables, cat_slope_table_out)
wtd_wet = arcpy.Merge_management(wtd_wet_taba_tables, wtd_wet_table_out)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)

print ('Tables merged')

In [33]:
#Set up field dictionary
elevDict = { 'ZONE_CODE': ('cat_elev_ZONE_CODE', 'wtd_elev_ZONE_CODE'),
         'COUNT': ('cat_elev_COUNT', 'wtd_elev_COUNT'),
          'AREA': ('cat_elev_AREA', 'wtd_elev_AREA'),
          'MIN': ('cat_elev_MIN', 'wtd_elev_MIN'),
          'MAX': ('cat_elev_MAX', 'wtd_elev_MAX'),
          'RANGE': ('cat_elev_RANGE', 'wtd_elev_RANGE'),
          'MEAN': ('cat_elev_MEAN', 'wtd_elev_MEAN'),
          'STD': ('cat_elev_STD', 'wtd_elev_STD'),
          'SUM': ('cat_elev_SUM', 'wtd_elev_SUM'),
          'VARIETY': ('cat_elev_VARIETY', 'wtd_elev_VARIETY'),
          'MAJORITY': ('cat_elev_MAJORITY', 'wtd_elev_MAJORITY'),
          'MINORITY': ('cat_elev_MINORITY', 'wtd_elev_MINORITY'),
          'MEDIAN': ('cat_elev_MEDIAN', 'wtd_elev_MEDIAN'),
          'PCT90': ('cat_elev_PCT90', 'wtd_elev_PCT90')
         }

slopeDict = { 'ZONE_CODE': ('cat_slope_ZONE_CODE', 'wtd_slope_ZONE_CODE'),
         'COUNT': ('cat_slope_COUNT', 'wtd_slope_COUNT'),
          'AREA': ('cat_slope_AREA', 'wtd_slope_AREA'),
          'MIN': ('cat_slope_MIN', 'wtd_slope_MIN'),
          'MAX': ('cat_slope_MAX', 'wtd_slope_MAX'),
          'RANGE': ('cat_slope_RANGE', 'wtd_slope_RANGE'),
          'MEAN': ('cat_slope_MEAN', 'wtd_slope_MEAN'),
          'STD': ('cat_slope_STD', 'wtd_slope_STD'),
          'SUM': ('cat_slope_SUM', 'wtd_slope_SUM'),
          'VARIETY': ('cat_slope_VARIETY', 'wtd_slope_VARIETY'),
          'MAJORITY': ('cat_slope_MAJORITY', 'wtd_slope_MAJORITY'),
          'MINORITY': ('cat_slope_MINORITY', 'wtd_slope_MINORITY'),
          'MEDIAN': ('cat_slope_MEDIAN', 'wtd_slope_MEDIAN'),
          'PCT90': ('cat_slope_PCT90', 'wtd_slope_PCT90')
         }

# Rename fields for elevation tables
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][1]
        newalias = elevDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][0]
        newalias = elevDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)

for field in arcpy.ListFields(cat_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][0]
        newalias = slopeDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_slope, keyval, newname, newalias)



ZONE_CODE wtd_elev_ZONE_CODE
COUNT wtd_elev_COUNT
AREA wtd_elev_AREA
MIN wtd_elev_MIN
MAX wtd_elev_MAX
RANGE wtd_elev_RANGE
MEAN wtd_elev_MEAN
STD wtd_elev_STD
SUM wtd_elev_SUM
VARIETY wtd_elev_VARIETY
MAJORITY wtd_elev_MAJORITY
MINORITY wtd_elev_MINORITY
MEDIAN wtd_elev_MEDIAN
PCT90 wtd_elev_PCT90
ZONE_CODE cat_elev_ZONE_CODE
COUNT cat_elev_COUNT
AREA cat_elev_AREA
MIN cat_elev_MIN
MAX cat_elev_MAX
RANGE cat_elev_RANGE
MEAN cat_elev_MEAN
STD cat_elev_STD
SUM cat_elev_SUM
VARIETY cat_elev_VARIETY
MAJORITY cat_elev_MAJORITY
MINORITY cat_elev_MINORITY
MEDIAN cat_elev_MEDIAN
PCT90 cat_elev_PCT90
ZONE_CODE wtd_slope_ZONE_CODE
COUNT wtd_slope_COUNT
AREA wtd_slope_AREA
MIN wtd_slope_MIN
MAX wtd_slope_MAX
RANGE wtd_slope_RANGE
MEAN wtd_slope_MEAN
STD wtd_slope_STD
SUM wtd_slope_SUM
MEDIAN wtd_slope_MEDIAN
PCT90 wtd_slope_PCT90
ZONE_CODE cat_slope_ZONE_CODE
COUNT cat_slope_COUNT
AREA cat_slope_AREA
MIN cat_slope_MIN
MAX cat_slope_MAX
RANGE cat_slope_RANGE
MEAN cat_slope_MEAN
STD cat_slope_STD


## Convert tables to pandas df and explore results
* **Merge tables and export to csv**

In [34]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places

In [35]:
# Make catchment elev df
cat_df = pd.DataFrame()
cat_field_list = []
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
    print (field.name, field.aliasName)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","cat_elev_ZONE_CODE"],axis=1)
cat_df = cat_df.set_index('cat_ID_txt')
cat_df

OBJECTID OBJECTID
cat_ID_txt cat_ID_txt
cat_elev_ZONE_CODE cat_elev_ZONE_CODE
cat_elev_COUNT cat_elev_COUNT
cat_elev_AREA cat_elev_AREA
cat_elev_MIN cat_elev_MIN
cat_elev_MAX cat_elev_MAX
cat_elev_RANGE cat_elev_RANGE
cat_elev_MEAN cat_elev_MEAN
cat_elev_STD cat_elev_STD
cat_elev_SUM cat_elev_SUM
cat_elev_VARIETY cat_elev_VARIETY
cat_elev_MAJORITY cat_elev_MAJORITY
cat_elev_MINORITY cat_elev_MINORITY
cat_elev_MEDIAN cat_elev_MEDIAN
cat_elev_PCT90 cat_elev_PCT90
region region


,cat_elev_COUNT,cat_elev_AREA,cat_elev_MIN,cat_elev_MAX,cat_elev_RANGE,cat_elev_MEAN,cat_elev_STD,cat_elev_SUM,cat_elev_VARIETY,cat_elev_MAJORITY,cat_elev_MINORITY,cat_elev_MEDIAN,cat_elev_PCT90,region
cat_ID_txt,,,,,,,,,,,,,,
48267,7607.00,760700.00,9,48,39,30.06,8.90,228638.00,40,34,9,32,40,Kodiak
49617,1470.00,147000.00,12,54,42,19.59,9.47,28800.00,43,15,34,16,30,Kodiak
50197,819.00,81900.00,13,35,22,21.07,5.38,17260.00,23,14,33,21,28,Kodiak
64593,2080.00,208000.00,13,75,62,45.99,13.43,95652.00,63,49,14,47,63,Kodiak
72144,657.00,65700.00,38,121,83,80.80,19.10,53086.00,84,74,38,79,106,Kodiak
76954,4168.00,416800.00,29,80,51,41.13,8.84,171422.00,52,33,29,38,54,Kodiak
77794,4689.00,468900.00,15,88,73,27.75,10.86,130100.00,74,21,83,24,38,Kodiak
90346,19618.00,1961800.00,81,638,557,183.12,112.56,3592436.00,546,81,513,140,353,Kodiak
93176,853.00,85300.00,123,195,72,140.28,16.51,119656.00,67,131,181,133,168,Kodiak


In [51]:
# Make catchment slope df
cat_sl_df = pd.DataFrame()
cat_sl_field_list = []
for field in arcpy.ListFields(cat_slope):
    cat_sl_field_list.append(field.name)
    print (field.name, field.aliasName)
cat_sl_arr = arcpy.da.TableToNumPyArray(cat_slope, cat_sl_field_list)
cat_sl_df = pd.DataFrame(cat_sl_arr)
cat_sl_df = cat_sl_df.drop(["OBJECTID", "cat_slope_ZONE_CODE"],axis=1)
cat_sl_df = cat_sl_df.set_index('cat_ID_txt')
cat_sl_df

OBJECTID OBJECTID
cat_ID_txt cat_ID_txt
cat_slope_ZONE_CODE cat_slope_ZONE_CODE
cat_slope_COUNT cat_slope_COUNT
cat_slope_AREA cat_slope_AREA
cat_slope_MIN cat_slope_MIN
cat_slope_MAX cat_slope_MAX
cat_slope_RANGE cat_slope_RANGE
cat_slope_MEAN cat_slope_MEAN
cat_slope_STD cat_slope_STD
cat_slope_SUM cat_slope_SUM
cat_slope_MEDIAN cat_slope_MEDIAN
cat_slope_PCT90 cat_slope_PCT90
region region


,cat_slope_COUNT,cat_slope_AREA,cat_slope_MIN,cat_slope_MAX,cat_slope_RANGE,cat_slope_MEAN,cat_slope_STD,cat_slope_SUM,cat_slope_MEDIAN,cat_slope_PCT90,region
cat_ID_txt,,,,,,,,,,,
48267,7607.00,760700.00,0.00,0.37,0.37,0.06,0.07,421.61,0.03,0.16,Kodiak
49617,1470.00,147000.00,0.00,0.27,0.27,0.05,0.05,68.74,0.03,0.10,Kodiak
50197,819.00,81900.00,0.00,0.19,0.19,0.06,0.04,49.71,0.05,0.12,Kodiak
64593,2080.00,208000.00,0.00,0.65,0.65,0.19,0.13,388.04,0.15,0.39,Kodiak
72144,657.00,65700.00,0.01,0.32,0.31,0.14,0.06,92.46,0.14,0.22,Kodiak
76954,4168.00,416800.00,0.00,0.47,0.47,0.06,0.07,265.16,0.04,0.16,Kodiak
77794,4689.00,468900.00,0.00,0.47,0.47,0.06,0.07,301.42,0.03,0.17,Kodiak
90346,19618.00,1961800.00,0.00,0.75,0.75,0.17,0.14,3360.34,0.14,0.40,Kodiak
93176,853.00,85300.00,0.00,0.22,0.22,0.07,0.05,55.72,0.05,0.14,Kodiak


In [36]:
# Make watershed elev df
wtd_df = pd.DataFrame()
wtd_field_list = []
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)
wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","wtd_elev_ZONE_CODE"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_txt')
wtd_df

,wtd_elev_COUNT,wtd_elev_AREA,wtd_elev_MIN,wtd_elev_MAX,wtd_elev_RANGE,wtd_elev_MEAN,wtd_elev_STD,wtd_elev_SUM,wtd_elev_VARIETY,wtd_elev_MAJORITY,wtd_elev_MINORITY,wtd_elev_MEDIAN,wtd_elev_PCT90,region
cat_ID_txt,,,,,,,,,,,,,,
48267,809610.00,80961000.00,-15,709,724,168.46,144.00,136387661.00,708,19,-15,124,389,Kodiak
49617,2945358.00,294535800.00,12,1108,1096,367.98,202.76,1083822543.00,1096,126,1090,335,660,Kodiak
50197,494751.00,49475100.00,13,800,787,216.73,169.23,107229713.00,787,33,797,168,469,Kodiak
64593,640989.00,64098900.00,-18,695,713,158.48,126.71,101585822.00,694,26,-18,112,358,Kodiak
72144,46117.00,4611700.00,38,533,495,249.93,93.57,11525983.00,461,164,38,238,380,Kodiak
76954,312795.00,31279500.00,18,819,801,271.13,176.52,84809560.00,794,33,25,241,528,Kodiak
77794,552014.00,55201400.00,15,819,804,241.40,175.70,133254385.00,802,33,814,210,504,Kodiak
90346,79315.00,7931500.00,81,761,680,313.71,158.83,24881881.00,681,81,761,285,555,Kodiak
93176,109901.00,10990100.00,104,897,793,479.47,173.15,52694292.00,784,296,110,481,715,Kodiak


In [52]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
    print (field.name, field.aliasName)
wtd_sl_arr = arcpy.da.TableToNumPyArray(cat_slope, cat_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID", "wtd_slope_ZONE_CODE"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_txt')
wtd_sl_df

OBJECTID OBJECTID
cat_ID_txt cat_ID_txt
wtd_slope_ZONE_CODE wtd_slope_ZONE_CODE
wtd_slope_COUNT wtd_slope_COUNT
wtd_slope_AREA wtd_slope_AREA
wtd_slope_MIN wtd_slope_MIN
wtd_slope_MAX wtd_slope_MAX
wtd_slope_RANGE wtd_slope_RANGE
wtd_slope_MEAN wtd_slope_MEAN
wtd_slope_STD wtd_slope_STD
wtd_slope_SUM wtd_slope_SUM
wtd_slope_MEDIAN wtd_slope_MEDIAN
wtd_slope_PCT90 wtd_slope_PCT90
region region


KeyError: "['wtd_slope_ZONE_CODE'] not found in axis"

In [37]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_txt')
wtd_n_df

,VALUE_0,VALUE_1,region,wtd_north_per,cat_ID_con
cat_ID_txt,,,,,
48267,59374100.00,21577800.00,Kodiak,26.66,Kodiak_48267
49617,221868200.00,72658500.00,Kodiak,24.67,Kodiak_49617
50197,36995200.00,12470800.00,Kodiak,25.21,Kodiak_50197
64593,46729200.00,17231700.00,Kodiak,26.94,Kodiak_64593
72144,3194700.00,1407900.00,Kodiak,30.59,Kodiak_72144
76954,20241600.00,11028800.00,Kodiak,35.27,Kodiak_76954
77794,40408000.00,14784300.00,Kodiak,26.79,Kodiak_77794
90346,5497500.00,2424900.00,Kodiak,30.61,Kodiak_90346
93176,7172100.00,3808900.00,Kodiak,34.69,Kodiak_93176


In [46]:
# Make watershed wetland df
wtd_wet_df = pd.DataFrame()
wtd_wet_field_list = []
for field in arcpy.ListFields(wtd_wet):
    wtd_wet_field_list.append(field.name)
wtd_wet_arr = arcpy.da.TableToNumPyArray(wtd_wet,wtd_wet_field_list)
wtd_wet_df = pd.DataFrame(wtd_wet_arr)
wtd_wet_df = wtd_wet_df.drop("OBJECTID",axis=1)
wtd_wet_df = wtd_wet_df.set_index('cat_ID_txt')
wtd_wet_df

,VALUE_0,VALUE_1,region,wtd_wet_per,cat_ID_con
cat_ID_txt,,,,,
48267,77265700.00,3721000.00,Kodiak,4.59,Kodiak_48267
49617,270940400.00,23621100.00,Kodiak,8.02,Kodiak_49617
50197,48019400.00,1481400.00,Kodiak,2.99,Kodiak_50197
64593,64991900.00,1004300.00,Kodiak,1.52,Kodiak_64593
72144,4627500.00,9900.00,Kodiak,0.21,Kodiak_72144
76954,31140100.00,165100.00,Kodiak,0.53,Kodiak_76954
77794,54843900.00,383200.00,Kodiak,0.69,Kodiak_77794
90346,7686300.00,270900.00,Kodiak,3.40,Kodiak_90346
93176,11006200.00,9600.00,Kodiak,0.09,Kodiak_93176


In [47]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop("OBJECTID",axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_txt')
wtd_lp_df

,FType,wtd_lake_area_sqm,wtd_lake_per,region,cat_ID,cat_ID_con
cat_ID_txt,,,,,,
100826,390,2344208.79,4.66,Kodiak,100826,Kodiak_100826
101556,390,50385.29,0.11,Kodiak,101556,Kodiak_101556
103096,390,225741.31,0.23,Kodiak,103096,Kodiak_103096
103196,390,14728932.21,11.44,Kodiak,103196,Kodiak_103196
103296,390,720676.91,0.61,Kodiak,103296,Kodiak_103296
103456,390,1318629.14,1.10,Kodiak,103456,Kodiak_103456
103496,390,570500.73,0.53,Kodiak,103496,Kodiak_103496
106626,390,33017596.25,9.69,Kodiak,106626,Kodiak_106626
106676,390,33017596.25,9.68,Kodiak,106676,Kodiak_106676


In [49]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop("OBJECTID",axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_txt')
wtd_glac_df


,O1Region,wtd_glacier_area_sqm,wtd_glacier_per,region,cat_ID,cat_ID_con
cat_ID_txt,,,,,,
103496,1,6184287.50,5.71,Kodiak,103496,Kodiak_103496
128685,1,749596.32,0.39,Kodiak,128685,Kodiak_128685


In [ ]:
landmet_merge_df = pd.merge(wtd_df,cat_df,on='cat_ID_txt', how='outer')

In [ ]:
dfs = [wtd_df,cat_df]
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_txt'), dfs)

In [ ]:
df_final